In [6]:
from compChem.Hartree_Fock import Molecule
from scipy.linalg import eigh
import numpy as np
import psi4

In [7]:
def basischanger(c, f):
        """
        allows a change in basis
        
        input:
        c: transformation matrix
        f: operator to be transformed
        """
        return np.einsum("ij,jm,mn->in", c.T, f, c, optimize=True)

In [8]:
class ConstrainedMolecule:
    def __init__(self, geom_file):
        """initiation method, will take in a geometry"""
        #this parameter contains all parameters and methods from the Molecule class
        self.id = Molecule(geom_file)
        self.id.setGuess()
        
        #getting some old and some new matrices
        self.gamma_alpha = self.id.getDensityMatrix("alpha")
        self.gamma_beta = self.id.getDensityMatrix("beta")
        self.pMatrix = (self.gamma_alpha + self.gamma_beta)/2
        self.transform = eigh(self.pMatrix, self.id.overlap)[1]
        
    
    def fock_Alternator(self, spin):
        "alters a fock operator to follow CUHF"
        assert spin == "alpha" or spin == "beta", f"{spin}: no valid spin"
        f = self.id.displayFockMatrix(spin)
        f_trans = basischanger(self.transform, f)
        coulomb_a = np.einsum("nopq,pq->no", self.id.displayElectronRepulsion(), self.pMatrix, optimize=True)
        exchange = np.einsum("npoq,pq->no", self.id.displayElectronRepulsion(), self.pMatrix, optimize=True)
        closed_shell_type = 0.5*(self.id.displayHamiltonian() + 2*coulomb_a - exchange)
        closed_shell_trans = basischanger(self.transform, closed_shell_type)
        
        # amount of paired electrons
        cc = self.id.beta
        
        # amount of unpaored electrons
        oo = self.id.alpha - self.id.beta
        
        # amount of unoccupied orbitals
        
        # alter first block
        f_trans[:cc + oo, cc + oo:] = closed_shell_type[:cc + oo, cc + oo:]
        
        # alter block in transposed position
        f_trans[cc + oo:, :cc + oo] = closed_shell_type[cc + oo:, :cc + oo]
        
        
        return f_trans
        
    def iteratinator(self):
        self.id.setGuess(self.fock_Alternator("alpha"), "alpha")
        self.id.setGuess(self.fock_Alternator("beta"), "beta")
        return self.id.iterator(iteration=1000)
    
    
    def updatePMatrix(self):
        """updates pMatrix to current self.id densitymatrices"""
        self.pMatrix = (self.id.getDensityMatrix("alpha") + self.id.getDensityMatrix("beta"))

In [25]:
psi4.set_options({"BASIS": "cc-pvdz"})
h1 = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
h1.id.setConvergence(1e-10)
h1.iteratinator()

/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:56: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.guessMatrix_a == "empty" and self.guessMatrix_b == "empty":
/user/gent/440/vsc44013/.local/lib/python3.7/site-packages/compChem/Hartree_Fock.py:210: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  assert self.guessMatrix_a != "empty" and self.guessMatrix_b != "empty", "make a guess first"


iteration: 0, E_tot:  2.90031505, E_elek:  1.31278341, deltaE:  1.31278341, rmsD:  28.12184906
iteration: 1, E_tot: -1.42987875, E_elek: -3.01741038, deltaE: -4.33019379, rmsD:  3.31696294
iteration: 2, E_tot: -1.49812188, E_elek: -3.08565351, deltaE: -0.06824313, rmsD:  0.89634096
iteration: 3, E_tot: -1.50394038, E_elek: -3.09147201, deltaE: -0.00581850, rmsD:  0.33358170
iteration: 4, E_tot: -1.50491018, E_elek: -3.09244181, deltaE: -0.00096981, rmsD:  0.14277933
iteration: 5, E_tot: -1.50518207, E_elek: -3.09271370, deltaE: -0.00027189, rmsD:  0.06456314
iteration: 6, E_tot: -1.50528610, E_elek: -3.09281773, deltaE: -0.00010403, rmsD:  0.04828107
iteration: 7, E_tot: -1.50533541, E_elek: -3.09286704, deltaE: -0.00004931, rmsD:  0.05593426
iteration: 8, E_tot: -1.50536429, E_elek: -3.09289592, deltaE: -0.00002888, rmsD:  0.06421803
iteration: 9, E_tot: -1.50538524, E_elek: -3.09291687, deltaE: -0.00002095, rmsD:  0.07069180
iteration: 10, E_tot: -1.50540320, E_elek: -3.09293483, del

iteration: 110, E_tot: -1.50626928, E_elek: -3.09380091, deltaE: -0.00000033, rmsD:  0.01078417
iteration: 111, E_tot: -1.50626959, E_elek: -3.09380123, deltaE: -0.00000031, rmsD:  0.01043979
iteration: 112, E_tot: -1.50626989, E_elek: -3.09380152, deltaE: -0.00000029, rmsD:  0.01010635
iteration: 113, E_tot: -1.50627016, E_elek: -3.09380180, deltaE: -0.00000028, rmsD:  0.00978352
iteration: 114, E_tot: -1.50627042, E_elek: -3.09380206, deltaE: -0.00000026, rmsD:  0.00947096
iteration: 115, E_tot: -1.50627067, E_elek: -3.09380230, deltaE: -0.00000024, rmsD:  0.00916837
iteration: 116, E_tot: -1.50627089, E_elek: -3.09380253, deltaE: -0.00000023, rmsD:  0.00887541
iteration: 117, E_tot: -1.50627111, E_elek: -3.09380274, deltaE: -0.00000021, rmsD:  0.00859181
iteration: 118, E_tot: -1.50627131, E_elek: -3.09380294, deltaE: -0.00000020, rmsD:  0.00831726
iteration: 119, E_tot: -1.50627150, E_elek: -3.09380313, deltaE: -0.00000019, rmsD:  0.00805147
iteration: 120, E_tot: -1.50627167, E_el

iteration: 220, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00030619
iteration: 221, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00029646
iteration: 222, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00028704
iteration: 223, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00027791
iteration: 224, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00026908
iteration: 225, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00026053
iteration: 226, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00025225
iteration: 227, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00024423
iteration: 228, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00023647
iteration: 229, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00022896
iteration: 230, E_tot: -1.50627432, E_el

iteration: 331, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000850
iteration: 332, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000823
iteration: 333, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000797
iteration: 334, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000772
iteration: 335, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000747
iteration: 336, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000723
iteration: 337, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000700
iteration: 338, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000678
iteration: 339, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000657
iteration: 340, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000636
iteration: 341, E_tot: -1.50627432, E_el

iteration: 442, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000024
iteration: 443, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000023
iteration: 444, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000022
iteration: 445, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000021
iteration: 446, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000021
iteration: 447, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000020
iteration: 448, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000019
iteration: 449, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000019
iteration: 450, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000018
iteration: 451, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000018
iteration: 452, E_tot: -1.50627432, E_el

iteration: 553, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 554, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 555, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 556, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 557, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 558, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000001
iteration: 559, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 560, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000001
iteration: 561, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000001
iteration: 562, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 563, E_tot: -1.50627432, E_el

iteration: 664, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 665, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 666, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 667, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 668, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 669, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 670, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 671, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 672, E_tot: -1.50627432, E_elek: -3.09380595, deltaE: -0.00000000, rmsD:  0.00000000
iteration: 673, E_tot: -1.50627432, E_elek: -3.09380595, deltaE:  0.00000000, rmsD:  0.00000000
iteration: 674, E_tot: -1.50627432, E_el

-1.5062743202988242

In [10]:
h_test = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")


In [26]:
p = h_test.pMatrix


1.1358984016271017

In [30]:
eigh(p, h_test.id.overlap)

(array([-3.66075249e-16, -1.58170768e-16, -1.24045094e-16, -3.93779042e-17,
        -6.61439556e-18, -4.98527348e-33, -5.44330304e-49,  9.55208764e-33,
         1.90320645e-26,  1.03876299e-16,  1.47615082e-16,  5.97286359e-16,
         8.23666094e-16,  7.36327900e-01,  2.77930005e+00]),
 array([[-8.63648649e-01, -1.21606884e-01,  1.83475091e-01,
          1.32347781e-01,  5.50411660e-03,  1.36318678e-12,
          9.99860711e-18, -1.95375976e-13,  8.11846955e-06,
          1.34994468e-01,  1.12745021e-01, -7.22899251e-01,
         -5.60295725e-01,  7.80864912e-01, -1.07977346e+00],
        [ 1.25707458e+00,  2.36500090e-02, -3.03280637e-01,
         -3.24260388e-01,  1.71195140e-01,  8.49864047e-13,
          6.31954454e-18, -1.22016552e-13,  5.05997406e-06,
         -1.21938652e-01, -3.23182142e-02,  1.18295660e+00,
          6.37624291e-01, -4.61038779e-01, -1.47538216e+00],
        [-3.45690588e-01, -6.71451891e-03, -1.40559604e-01,
         -7.13538208e-03,  3.74098057e-01,  2.190

In [27]:
a = h_test.id.getDensityMatrix("alpha")

In [28]:
a.trace()

1.79992851287294